In [71]:
import sys
import itertools
import json
import re
import warnings
import numpy as np
import pandas as pd

from tqdm import tqdm
from six import string_types
from copy import deepcopy
from ase.io import read

from pymatgen.core import Composition, Structure, Element
from pymatgen.core.ion import Ion

from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.analysis.phase_diagram import PhaseDiagram
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, \
PourbaixEntry, IonEntry, ELEMENTS_HO, CAPITALIZM HO
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.entries.compatibility import MaterialsProjectCompatibility, \
MaterialsProjectAqueousCompatibility

from pymatgen.ext.matproj import MPRester



# from pymatgen.core.periodic_table import Element
# from pymatgen.core.structure import Structure


mpr = MPRester('rOpOm94QqJRPQUba0X')

__version__

/home/ahrehd0506/miniconda3/envs/ocp-models/lib/python3.9/site-packages/pymatgen/ext/matproj.py:183: UserWarning: You are using the legacy MPRester. This version of the MPRester will no longer be updated. To access the latest data with the new MPRester, obtain a new API key from https://materialsproject.org/api and consult the docs at https://docs.materialsproject.org/ for more information.
  warnings.warn(


'2023.5.10'

## Define function to calculate stabilities with own DFT calculation results

In [69]:
def traj_to_computed_entry(path):
    """
    It converts our own calculations to Computed Entries
    
    Corrections for oxides 
    taken from "https://github.com/materialsproject/pymatgen/blob/7461cfccd32f239291bd50d2b0aebae2cd8088e5/pymatgen/entries/MPCompatibility.yaml"
    """  

    correction_dict={}

    # Correction for O is OxideCorrection in the above url
    correction_dict['O'] = -0.70229

    # Correction for metals below is UCorrection (Advanced)
    correction_dict['Mn'] = -1.68085015096   #Fit to MnO, Mn3O4 and MnO2 (BURP:-1.687)
    correction_dict['Fe'] = -2.733           #Fit to FeO and Fe2O3 (Fe3O4 probably wrong)
    correction_dict['Co'] = -1.874           #Fit to CoO, Co3O4 (BURP:-1.751)
    correction_dict['Cr'] = -2.013           #Fit to Cr2O3 (CrO3 missing) (BURP: -2.067)
    correction_dict['Mo'] = -3.531           #Fit to MoO3 and MoO2 (BURP: -2.668)
    correction_dict['W']=  -4.351           #Fit to WO2 and WO3 (BURP: -2.762)
    correction_dict['V'] =  -1.682           #Fit to V2O3 and V2O5 (VO2 fit is way off) (BURP: -1.764)
    correction_dict['Ni'] = -2.164           #Based on burp version as of Feb 28 2011

    """
    Make sure all calculations are performed with 
    MATERIALS PROJECTS calculation settings (especially Pseudopotential, U corrections and xc='PBE')
    """

    atoms = read(path)
    energy = atoms.get_potential_energy()
    comp = Composition(str(atoms.symbols))
    comp_dict = comp.as_dict()

    calc_dict = atoms.get_calculator().todict()
    ldau_elements = list(calc_dict['ldau_luj'].keys())

    # Confirm if it used GGA+U or not
    if any([i for i in comp_dict.keys() if i in ldau_elements]) and calc_dict['ldau']:
        lda_use = True
    else:
        lda_use = False

    # Calculate corrections
    # Only elements that need corrections 
    correction_element_list = [i for i in comp_dict.keys() if i in correction_dict.keys()]
    correction = 0
    for i in correction_element_list:
        correction += correction_dict[i]*comp_dict[i]

    # Add parameters
    parameters={}
    parameters['oxide_type']='oxide'
    parameters['pseudo_potential'] = {}
    parameters['pseudo_potential']['functional'] = 'PBE'
    parameters['pseudo_potential']['pot_type'] = 'paw'

    # Pseudopotential related
    pp = []
    for elem in comp_dict.keys():
        if elem != 'O':
            if elem in ['Bi','Te','Si','B','Au','Sb']:
                pp.append(str(elem))
            else:
                pp.append(str(elem)+str(calc_dict['setups'][elem]))
        else: 
            pp.append('O')

    parameters['pseudo_potential']['labels'] = pp
    parameters['potcar_symbols'] = []
    for p in pp:
        parameters['potcar_symbols'].append('PBE '+str(p))

    # U related
    if lda_use == True:
        parameters['run_type'] = 'GGA+U'
        parameters['is_hubbard'] = True
    else: 
        parameters['run_type'] = 'GGA'
        parameters['is_hubbard'] = False

    parameters['hubbards'] = {}
    for elem in comp_dict.keys():
        if elem in ldau_elements:
            parameters['hubbards'][elem] = calc_dict['ldau_luj'][elem]

    data={}
    data['oxide_type']='oxide'
    
    return ComputedEntry(composition=comp, 
                         energy=energy, 
                         correction=correction, 
                         parameters=parameters, 
                         data=data, 
                         entry_id='sback')


def get_ion_ref(mpr, chemsys):
    """
      This is to collect reference states. It is taken from "get_pourbaix_entries"
    """
    """
    
    A helper function to get all entries necessary to generate
    a pourbaix diagram from the rest interface.    

    Args:

        chemsys ([str]): A list of elements comprising the chemical system, e.g. ['Li', 'Fe']
    """

    pbx_entries = []
    # Get ion entries first, because certain ions have reference
    # solids that aren't necessarily in the chemsys (Na2SO4)
    url = '/pourbaix_diagram/reference_data/' + '-'.join(chemsys)
    ion_data = mpr._make_request(url)
    ion_ref_comps = [Composition(d['Reference Solid']) for d in ion_data]
    ion_ref_elts = list(itertools.chain.from_iterable(i.elements for i in ion_ref_comps))
    ion_ref_entries = mpr.get_entries_in_chemsys(
            list(set([str(e) for e in ion_ref_elts] + ['O', 'H'])),
            property_data=['e_above_hull'], compatible_only=False)
    
    
    # suppress the warning about supplying the required energies; they will be calculated from the
    # entries we get from MPRester
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="You did not provide the required O2 and H2O energies.")
        compat = MaterialsProjectAqueousCompatibility(solid_compat=MaterialsProjectCompatibility())
    ion_ref_entries = compat.process_entries(ion_ref_entries)
    ion_ref_pd = PhaseDiagram(ion_ref_entries)

    # position the ion energies relative to most stable reference state
    for n, i_d in enumerate(ion_data):
        ion = Ion.from_formula(i_d['Name'])
        refs = [e for e in ion_ref_entries
                if e.composition.reduced_formula == i_d['Reference Solid']]
        if not refs:
            raise ValueError("Reference solid not contained in entry list")
        stable_ref = sorted(refs, key=lambda x: x.data['e_above_hull'])[0]
        rf = stable_ref.composition.get_reduced_composition_and_factor()[1]
        solid_diff = ion_ref_pd.get_form_energy(stable_ref) - i_d['Reference solid energy'] * rf
        elt = i_d['Major_Elements'][0]
        correction_factor = ion.composition[elt] / stable_ref.composition[elt]
        energy = i_d['Energy'] + solid_diff * correction_factor
        ion_entry = IonEntry(ion, energy)
        pbx_entries.append(PourbaixEntry(ion_entry, 'ion-{}'.format(n)))
        
        
    return pbx_entries, ion_ref_entries, ion_ref_pd

def get_pourbaix_entry_solid(entry,ion_ref_pd):
    """
    This is to calculate formation energies and convert Computed Entries to Pourbaix Entries
    """
    form_e = ion_ref_pd.get_form_energy(entry)
    computed_entry = deepcopy(entry)
    new_entry = ComputedEntry(computed_entry.composition, form_e, entry_id=computed_entry.entry_id)
    pbx_entry = PourbaixEntry(new_entry)    
    
    return pbx_entry

## Calculate PBX stability

In [ ]:
import collections
import os
import pickle
import multiprocess as mp
import tqdm.notebook as tqdm

from scipy.spatial.qhull import QhullError
from joblib import Memory

# Cache pourbaix_entries, most time intensive part
def calc_stability_from_pourbaix(path):
    stability = [] # acid_ORR, acid_OER
    try:
        computed_entry = traj_to_computed_entry(path)
        composition = computed_entry.composition
        comp_dict = {str(key): value for key, value in composition.items()
                if key not in ELEMENTS_HO}
        pourbaix_entries, ion_ref_entries, ion_ref_pd = get_ion_ref(mpr, [i for i in comp_dict.keys()])
        new_pourbaix_entry=get_pourbaix_entry_solid(computed_entry, ion_ref_pd)

        # Add new Pourbaix entries
        pourbaix_entries.append(new_pourbaix_entry)
        entry = [entry for entry in pourbaix_entries if entry.entry_id == 'sback'][0]
        pbx = PourbaixDiagram(pourbaix_entries, comp_dict = comp_dict, filter_solids = False)

        

        OER=max(pbx.get_decomposition_energy(entry, pH=0, V=1.3),
                pbx.get_decomposition_energy(entry, pH=0, V=1.4),
                pbx.get_decomposition_energy(entry, pH=0, V=1.5),
                pbx.get_decomposition_energy(entry, pH=0, V=1.6))
        ORR=max(pbx.get_decomposition_energy(entry, pH=0, V=0.8),
                pbx.get_decomposition_energy(entry, pH=0, V=0.9),
                pbx.get_decomposition_energy(entry, pH=0, V=1.0),
                pbx.get_decomposition_energy(entry, pH=0, V=1.1))
        stability.append([ORR, OER])
    except QhullError:
        stability.append([100,100])
    return stability

# Define a local cached version so that the result gets returned if the element combination 
# has already been considered
memory = Memory('./cachedir', verbose=0)
calc_stability_from_pourbaix_cached = memory.cache(calc_stability_from_pourbaix)

path = 'your_calculated_trajectory_path/'
name_list = [path + i for i in os.listdir(path)]

with mp.Pool(8) as pool:
    results = list(tqdm.tqdm(pool.imap(calc_stability_from_pourbaix_cached, name_list), total=len(name_list)))

In [ ]:
# Some cases, we have Qhull Error

qhull_error = []
for result, name in zip(results, name_list):
    if result[0][0] == 100:
        qhull_error.append(name)
        
print((qhull_error))

## Calculate energy above hull

In [ ]:
def e_above_hull(path):
    atoms = read(path)
    comp = Composition(str(atoms.symbols))
    comp_dict={}
    for i in comp:
        comp_dict[str(i)] = int(comp[i])
    all_elements = [i for i in comp_dict.keys()]
    entries=mpr.get_entries_in_chemsys(all_elements)
       
    # Ad newly calculated materials
    computed_entry = traj_to_computed_entry(path)
    entries.append(computed_entry)
    pd = PhaseDiagram(entries)
    return pd.get_e_above_hull([entry for entry in entries if entry.entry_id == 'sback'][0])

def calc_hull(path):
    return e_above_hull(path)

memory = Memory('./cachedir', verbose=0)
calc_hull_cached = memory.cache(calc_hull)

In [ ]:
with mp.Pool(4) as pool:
    hull_results = list(tqdm.tqdm(pool.imap(calc_hull_cached, name_list), total=len(name_list)))

## Merge data

In [ ]:
ORR_stability = []
OER_stability = []
hull = []
completed_name_list = []

for result, name, hull_e in zip(results, name_list, hull_results):
    if result[0][0] != 100:
        ORR_stability.append(result[0][0])
        OER_stability.append(result[0][1])
        completed_name_list.append(name)
        hull.append(hull_e)

df = pd.DataFrame()
df['id'] = completed_name_list
df['OER_stability'] = OER_stability
df['hull_energy'] = hull

print(len(df))

df.to_csv('../data/stability_data.csv')